In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

from robmob import robot, sensors
from robmob.point_cloud import PointCloud
from time import sleep
import math

import numpy as np
import scipy
import scipy.ndimage

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('nbagg')
from matplotlib import cm

robot_ip = '192.168.0.112'
robot = robot.Robot(robot_ip)
robot.connect()

In [ ]:
depth_image = scipy.ndimage.imread("./img/depth_g.png") / 100.0 #Assume that depth_image contains distances in mm
height, width = depth_image.shape
# Intrinsic parameters:
# field of view
fov_x, fov_y = (365.456, 365.456)
# principal point
center_x, center_y = (254.878, 205.395)

In [ ]:

def to_points(depth_image):
    height, width = depth_image.shape

    theta_x = np.arctan((np.arange(width) - center_x) / fov_x)
    theta_y = np.arctan((np.arange(height) - center_y) / fov_y)
    theta_x = np.expand_dims(theta_x, axis=0) # Row matrix
    theta_y = np.expand_dims(theta_y, axis=1) # Column matrix

    dx = depth_image * np.cos(theta_x)
    dy = depth_image * np.cos(theta_y)

    x = dx * np.sin(theta_x)
    y = dy * np.sin(theta_y)
    z = depth_image

    x_column = np.reshape(x, (-1, 1))
    y_column = np.reshape(y, (-1, 1))
    z_column = np.reshape(z, (-1, 1))

    points = np.hstack((x_column, y_column, z_column))
    return points

### To view the point cloud

[link](point_cloud/viewer.html)

In [ ]:
depth_sensor = sensors.KinectDepthSensor()
rgb_sensor = sensors.KinectRGBSensor()
robot.add_sensor(depth_sensor)
robot.add_sensor(rgb_sensor)

In [ ]:
buffer_depth = depth_sensor.read_data()
buffer_rgb = rgb_sensor.read_data()
fig, (ax1, ax2) = plt.subplots(2, 1)

im1 = ax1.imshow(buffer_depth, aspect='equal')
im2 = ax2.imshow(buffer_rgb, aspect='equal')

cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im1, cax=cbar_ax)

plt.show()

In [ ]:
p = to_points(buffer_depth)
colors = np.asarray(buffer_rgb).reshape((-1, 3))

In [ ]:
PointCloud(p, colors).save()